In [14]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision.utils import save_image
import os
import copy
from PIL import Image
import numpy
import torch.optim as optim
import torch.nn.init as init
from torchvision import datasets, models, transforms
from torch.optim import lr_scheduler

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import os
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Custom Dataset Class
class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
      img_name = self.image_files[idx]
      label = int(img_name.split('_')[0]) - 1  # Convert to 0-indexed label
      img_path = os.path.join(self.folder_path, img_name)
      image = Image.open(img_path).convert("RGB")
      if self.transform:
          image = self.transform(image)
      return image, label


In [17]:
batch_size = 16
num_epochs = 30
num_workers = 0

criterion = nn.CrossEntropyLoss()

In [18]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
# Define the paths to your training and validation directories
train_dir = "/content/drive/MyDrive/project/train_data"  # Replace with the path to your training folder
val_dir = "/content/drive/MyDrive/project/val_data"      # Replace with the path to your validation folder

# Create datasets for training and validation
image_datasets = {
    'train': ImageDataset(train_dir, data_transforms['train']),
    'val': ImageDataset(val_dir, data_transforms['val'])
}

# Create dataloaders for training and validation
train_dl = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=num_workers),
    'val': DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=False, num_workers=num_workers)
}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [19]:
# model_ft = models.resnet18(weights = "IMAGENET1K_V1")
# model_ft.fc = nn.Linear(model_ft.fc.in_features,60)
# model_ft = model_ft.to(device)

# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm

device = torch.device('cpu')

# Initialize the model (ResNet18) and adjust the final layer for 60 classes
model_ft = models.resnet18(weights="IMAGENET1K_V1")
model_ft.fc = nn.Linear(model_ft.fc.in_features, 60)  # Adjusting for 60 classes
model_ft = model_ft.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Training parameters
num_epochs = 10
best_acc = 0.0

# Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print("-" * 10)

    # Loop over train and validation phases
    for phase in ['train', 'val']:
        if phase == 'train':
            model_ft.train()  # Set the model to training mode
        else:
            model_ft.eval()  # Set the model to evaluation mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over the data
        for inputs, labels in tqdm(train_dl[phase], desc=f"{phase} phase"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer_ft.zero_grad()  # Zero the parameter gradients

            # Forward pass
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model_ft(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward pass and optimize only in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer_ft.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Compute epoch loss and accuracy
        epoch_loss = running_loss / dataset_sizes[phase]  # Use dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]  # Use dataset_sizes[phase]

        print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Save the best model based on validation accuracy
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = model_ft.state_dict()

    print()

# Load the best model weights
model_ft.load_state_dict(best_model_wts)
print(f"Best Validation Accuracy: {best_acc:.4f}")

Epoch 1/10
----------


train phase: 100%|██████████| 188/188 [10:21<00:00,  3.31s/it]


Train Loss: 3.1933 Acc: 0.3063


val phase: 100%|██████████| 38/38 [06:06<00:00,  9.65s/it]


Val Loss: 1.6484 Acc: 0.6800

Epoch 2/10
----------


...

Val Loss: 0.1803 Acc: 0.9583

Epoch 10/10
----------


train phase: 100%|██████████| 188/188 [10:45<00:00,  3.43s/it]


Train Loss: 0.2951 Acc: 0.9380


val phase: 100%|██████████| 38/38 [00:43<00:00,  1.14s/it]

Val Loss: 0.1590 Acc: 0.9700

Best Validation Accuracy: 0.9700


In [24]:
best_model_path = "/content/drive/MyDrive/best_resnet_model.pth"

torch.save(best_model_wts, best_model_path)
print(f"Best model weights saved to {best_model_path}")

Best model weights saved to /content/drive/MyDrive/best_resnet_model.pth
